In [1]:
import os
import re
import pickle
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
# from keras.layers import Embedding
from  keras . utils  import  to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Permute
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Embedding

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd 
df_train=pd.read_csv('gdrive/My Drive/Thesis Data/Our dataset/ulm_train.csv')
df_test=pd.read_csv('gdrive/My Drive/Thesis Data/Our dataset/ulm_test.csv')


In [5]:
!git clone -l -s https://github.com/banglakit/bengali-stemmer.git

fatal: destination path 'bengali-stemmer' already exists and is not an empty directory.


In [6]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-b5hu25bj
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-b5hu25bj
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=fe194b7c1a6fc078863ec9e46e28974513a56c77f46f4854413470b15ba50e63
  Stored in directory: /tmp/pip-ephem-wheel-cache-r41lacxs/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [7]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-e7n90xd9
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-e7n90xd9
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=af4e8efb12528a4e5df961a153911c8b22eb13fa576be01382072135d10d5378
  Stored in directory: /tmp/pip-ephem-wheel-cache-d397xu33/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [8]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('বাংলায়')

'বাংলা'

In [9]:
total_data = df_train
total_data=total_data.append(df_test, ignore_index = True)

In [10]:
macronum=sorted(set(total_data['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

total_data['label']=total_data['label'].apply(fun)


In [11]:
texts = list(total_data['text'])
labels = list(total_data['label'])


In [12]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [13]:
stopwords = pd.read_csv('gdrive/My Drive/Colab Notebooks/Stopwords.csv')

In [15]:
set_stop = set(stopwords['words'])

In [16]:

def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if((cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [17]:
texts = pre_process(texts)

In [18]:
texts[0]

'সাম্প্রদায়িক সংঘাত মানবিক প্রতিরোধ সাম্প্রদায়িক সংঘাত মানবিক প্রতিরোধফকির ইলিয়াস দেশ থাকি বহুজাতিক বহুভাষিক মানুষ বাস ধর্মাবলম্বী মতাবলম্বী মানুষ কারো কারো মিল চাইল সামান্য বিষ প্রতিদিন দাঙ্গা তেমন হ্যাঁ প্রি পাঠক মার্কিন যুক্তরাষ্ট্র কথা বলছিএ দেশ মানুষ মানুষ বুক পাঁজর চিবি দেশ কঠোর আইন থাক পার পারব পারব কঠিন শাস্তির মুখোমুখি জীবনবাজি রায়ট কেপাক ভারত উপমহাদেশ জন্ম বিভক্ত রায়ট মধ্য কারোর অজানা জন্মইতিহাস জন্ম পাকিস্তান ভারত পরবর্তীকাল বাংলাদেশ ভূখণ্ড দ্বিজাতি তত্ত্ব শত্রু সম্পত্তি আইন মানুষ মানুষ ধর্ম বিভাজন সেভাব পরিকল্পিতভাব সাল ভারত ভাগ অসংখ্য হিন্দু ঘরবাড়ি ছেড় গিয় জিন্নাহর দ্বিজাতি তত্ত্ব ভিত্তি ওঠা দেশ অস্তিত্ব রক্ষা প্রশ্ন প্রথম সাল পূর্বতন পূর্ব পাকিস্তান সাম্প্রদায়িক দাঙ্গ হিন্দু নিরাপত্ত পালি সাল হিন্দু আওয়ামী লীগ ভোট বিশ্বাস দখলদ পাক সেনা সহযোগী প্রবল প্রতিশোধস্পৃহা খুঁজ খুঁজ হিন্দু নিধন চালি হিন্দু বুদ্ধিজীবী সমাজসেবী শ্রদ্ধে ব্যক্তিত্ব ঢাকা বিশ্ববিদ্যালয় জগন্নাথ সহস্রাধিক হিন্দু ছাত্র হত্যা সাল নভেম্বর তারিখ ইউএস সিনেট কমি প্রামাণ্য প্রতিবেদন সিনেটর এডওয়ার্ড কেনেডি লিখ “সব

In [19]:
def load_data(num_words, sequence_length, test_size=0.25, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes= 6
    y = to_categorical(np.asarray(labels), num_classes)
    tokenizer.fit_on_texts(texts)
    X = tokenizer.texts_to_sequences(texts)
    X = np.array(X)
    # pad sequences with 0's
    X = pad_sequences(X, maxlen=sequence_length)
    # split data to training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data

In [20]:
d = set()
for s in texts:
    for ss in list(s.split()):
        d.add(ss)

In [21]:
len(d)

112947

In [22]:
max([len(s.split()) for s in texts]) # find the max length

11276

In [23]:
MAX_SEQUENCE_LENGTH = 11276    #max([len(s.split()) for s in texts]) 
MAX_NUM_WORDS = 109803 + 1
vocab_size = MAX_NUM_WORDS
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

data = load_data(MAX_NUM_WORDS , MAX_SEQUENCE_LENGTH,VALIDATION_SPLIT)

In [24]:
len(data['tokenizer'].word_index)

109803

In [25]:
max_words = 109803 + 1
max_len = 11276
# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
# sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

LSTM only

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,300,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(6,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

    

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 11276)]           0         
_________________________________________________________________
embedding (Embedding)        (None, 11276, 300)        32941200  
_________________________________________________________________
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 6)                

In [ ]:
model.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=64, epochs=10, verbose=2)
# history = m.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=128, epochs=10, verbose=2)

Epoch 1/10
27/27 - 25s - loss: 0.4124 - accuracy: 0.4095 - val_loss: 0.2939 - val_accuracy: 0.5357
Epoch 2/10
27/27 - 24s - loss: 0.2456 - accuracy: 0.6548 - val_loss: 0.2469 - val_accuracy: 0.6643
Epoch 3/10
27/27 - 24s - loss: 0.1652 - accuracy: 0.7988 - val_loss: 0.2091 - val_accuracy: 0.7167
Epoch 4/10
27/27 - 24s - loss: 0.0849 - accuracy: 0.9101 - val_loss: 0.2377 - val_accuracy: 0.6857
Epoch 5/10
27/27 - 24s - loss: 0.0369 - accuracy: 0.9679 - val_loss: 0.2665 - val_accuracy: 0.7048
Epoch 6/10
27/27 - 24s - loss: 0.0097 - accuracy: 0.9952 - val_loss: 0.3953 - val_accuracy: 0.6643
Epoch 7/10
27/27 - 24s - loss: 0.0055 - accuracy: 0.9958 - val_loss: 0.2495 - val_accuracy: 0.7548
Epoch 8/10
27/27 - 24s - loss: 0.0013 - accuracy: 0.9994 - val_loss: 0.3424 - val_accuracy: 0.7333
Epoch 9/10
27/27 - 24s - loss: 2.3357e-04 - accuracy: 1.0000 - val_loss: 0.4045 - val_accuracy: 0.7524
Epoch 10/10
27/27 - 24s - loss: 0.0180 - accuracy: 0.9923 - val_loss: 0.3270 - val_accuracy: 0.7238


LSTM with ATTENTION

In [ ]:
# don't need this one
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,300,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(6,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [26]:
SINGLE_ATTENTION_VECTOR = False
APPLY_ATTENTION_BEFORE_LSTM = False
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    # output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [30]:
def model_attention_applied_after_lstm():
    #inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,300,input_length=max_len)(inputs)
    
    lstm_units = 64
    #lstm_units =128
    lstm_out = LSTM(lstm_units, return_sequences=True)(layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(6, activation='softmax')(attention_mul)
    model = Model(inputs=[inputs], outputs=output)
    return model

In [31]:
from keras.layers import merge
from keras.layers import multiply
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

from keras.utils.vis_utils import plot_model

In [32]:
INPUT_DIM = 300
TIME_STEPS = max_len

In [33]:
m = model_attention_applied_after_lstm()


In [34]:
m.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 11276)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 11276, 300)   32941200    inputs[0][0]                     
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 11276, 64)    93440       embedding[0][0]                  
__________________________________________________________________________________________________
permute (Permute)               (None, 64, 11276)    0           lstm[0][0]                       
_______________________________________________________________________________________

In [ ]:
# using lstm unit = 128 and bs=32
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

m.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=32, epochs=10, verbose=2)


Epoch 1/10
53/53 - 196s - loss: 0.4506 - accuracy: 0.1619 - val_loss: 0.4506 - val_accuracy: 0.1619
Epoch 2/10
53/53 - 207s - loss: 0.4486 - accuracy: 0.1940 - val_loss: 0.4379 - val_accuracy: 0.2810
Epoch 3/10
53/53 - 206s - loss: 0.4284 - accuracy: 0.2893 - val_loss: 0.4136 - val_accuracy: 0.3690
Epoch 4/10
53/53 - 205s - loss: 0.3888 - accuracy: 0.4071 - val_loss: 0.3840 - val_accuracy: 0.4405
Epoch 5/10
53/53 - 207s - loss: 0.3234 - accuracy: 0.5310 - val_loss: 0.3901 - val_accuracy: 0.4000
Epoch 6/10
53/53 - 207s - loss: 0.2509 - accuracy: 0.6232 - val_loss: 0.3692 - val_accuracy: 0.4619
Epoch 7/10
53/53 - 205s - loss: 0.1714 - accuracy: 0.7679 - val_loss: 0.3735 - val_accuracy: 0.5024
Epoch 8/10
53/53 - 205s - loss: 0.1131 - accuracy: 0.8601 - val_loss: 0.3898 - val_accuracy: 0.5310
Epoch 9/10
53/53 - 205s - loss: 0.0738 - accuracy: 0.9149 - val_loss: 0.4272 - val_accuracy: 0.4857
Epoch 10/10
53/53 - 205s - loss: 0.0574 - accuracy: 0.9423 - val_loss: 0.4316 - val_accuracy: 0.5143

In [ ]:
# using lstm unit = 64 and 
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

m.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=64, epochs=10, verbose=2)


Epoch 1/10
27/27 - 55s - loss: 0.4506 - accuracy: 0.1548 - val_loss: 0.4506 - val_accuracy: 0.1619
Epoch 2/10
27/27 - 56s - loss: 0.4506 - accuracy: 0.1423 - val_loss: 0.4506 - val_accuracy: 0.1619
Epoch 3/10
27/27 - 57s - loss: 0.4503 - accuracy: 0.1696 - val_loss: 0.4497 - val_accuracy: 0.1619
Epoch 4/10
27/27 - 58s - loss: 0.4395 - accuracy: 0.2768 - val_loss: 0.4293 - val_accuracy: 0.3167
Epoch 5/10
27/27 - 58s - loss: 0.4275 - accuracy: 0.3167 - val_loss: 0.4218 - val_accuracy: 0.3143
Epoch 6/10
27/27 - 58s - loss: 0.4167 - accuracy: 0.3345 - val_loss: 0.4139 - val_accuracy: 0.3310
Epoch 7/10
27/27 - 58s - loss: 0.3926 - accuracy: 0.3863 - val_loss: 0.4002 - val_accuracy: 0.3667
Epoch 8/10
27/27 - 58s - loss: 0.3611 - accuracy: 0.4571 - val_loss: 0.3876 - val_accuracy: 0.4381
Epoch 9/10
27/27 - 58s - loss: 0.3297 - accuracy: 0.5423 - val_loss: 0.3904 - val_accuracy: 0.3952
Epoch 10/10
27/27 - 58s - loss: 0.2881 - accuracy: 0.6405 - val_loss: 0.3827 - val_accuracy: 0.4214


In [ ]:
# using lstm unit = 64
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

m.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=128, epochs=10, verbose=2)


Epoch 1/10
14/14 - 53s - loss: 0.4506 - accuracy: 0.1554 - val_loss: 0.4506 - val_accuracy: 0.1643
Epoch 2/10
14/14 - 51s - loss: 0.4506 - accuracy: 0.1673 - val_loss: 0.4506 - val_accuracy: 0.1643
Epoch 3/10
14/14 - 51s - loss: 0.4506 - accuracy: 0.1673 - val_loss: 0.4506 - val_accuracy: 0.1643
Epoch 4/10
14/14 - 51s - loss: 0.4506 - accuracy: 0.1673 - val_loss: 0.4506 - val_accuracy: 0.1643
Epoch 5/10
14/14 - 51s - loss: 0.4505 - accuracy: 0.1690 - val_loss: 0.4504 - val_accuracy: 0.2357
Epoch 6/10
14/14 - 51s - loss: 0.4448 - accuracy: 0.2464 - val_loss: 0.4377 - val_accuracy: 0.2667
Epoch 7/10
14/14 - 50s - loss: 0.4352 - accuracy: 0.2667 - val_loss: 0.4289 - val_accuracy: 0.2905
Epoch 8/10
14/14 - 49s - loss: 0.4097 - accuracy: 0.3149 - val_loss: 0.4124 - val_accuracy: 0.3000
Epoch 9/10
14/14 - 50s - loss: 0.3817 - accuracy: 0.3661 - val_loss: 0.4022 - val_accuracy: 0.3500
Epoch 10/10
14/14 - 50s - loss: 0.3490 - accuracy: 0.4244 - val_loss: 0.3940 - val_accuracy: 0.3333


In [35]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [37]:
# using lstm unit = 64, bs=32, using categorical crossentropy with callbacks
m1 = model_attention_applied_after_lstm()

m1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

m1.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=64,callbacks=[callback], epochs=14, verbose=2)


Epoch 1/14
27/27 - 55s - loss: 1.7920 - accuracy: 0.1661 - val_loss: 1.7918 - val_accuracy: 0.1643
Epoch 2/14
27/27 - 56s - loss: 1.7917 - accuracy: 0.1655 - val_loss: 1.7917 - val_accuracy: 0.1619
Epoch 3/14
27/27 - 57s - loss: 1.7699 - accuracy: 0.2101 - val_loss: 1.7127 - val_accuracy: 0.2667
Epoch 4/14
27/27 - 57s - loss: 1.6480 - accuracy: 0.2940 - val_loss: 1.5814 - val_accuracy: 0.3095
Epoch 5/14
27/27 - 57s - loss: 1.4809 - accuracy: 0.3696 - val_loss: 1.5866 - val_accuracy: 0.3286
Epoch 6/14
27/27 - 57s - loss: 1.3205 - accuracy: 0.4571 - val_loss: 1.5942 - val_accuracy: 0.3500
Epoch 7/14
27/27 - 58s - loss: 1.1318 - accuracy: 0.5899 - val_loss: 1.6514 - val_accuracy: 0.3738
Epoch 8/14
27/27 - 58s - loss: 0.9928 - accuracy: 0.6494 - val_loss: 1.7539 - val_accuracy: 0.3810
Epoch 9/14
27/27 - 58s - loss: 0.8967 - accuracy: 0.6774 - val_loss: 1.7892 - val_accuracy: 0.3810
Epoch 10/14
27/27 - 58s - loss: 0.8346 - accuracy: 0.6744 - val_loss: 1.8689 - val_accuracy: 0.3810
Epoch 11/

In [36]:
# using lstm unit = 64, bs=32, using categorical crossentropy with callbacks
m1 = model_attention_applied_after_lstm()

m1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

m1.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=32,callbacks=[callback], epochs=14, verbose=2)


Epoch 1/14
53/53 - 70s - loss: 1.7920 - accuracy: 0.1554 - val_loss: 1.7917 - val_accuracy: 0.1714
Epoch 2/14
53/53 - 70s - loss: 1.7762 - accuracy: 0.2077 - val_loss: 1.7204 - val_accuracy: 0.2690
Epoch 3/14
53/53 - 70s - loss: 1.5718 - accuracy: 0.3149 - val_loss: 1.6318 - val_accuracy: 0.3214
Epoch 4/14
53/53 - 71s - loss: 1.3805 - accuracy: 0.4327 - val_loss: 1.5076 - val_accuracy: 0.3929
Epoch 5/14
53/53 - 72s - loss: 1.0725 - accuracy: 0.5655 - val_loss: 1.4733 - val_accuracy: 0.3905
Epoch 6/14
53/53 - 72s - loss: 0.8299 - accuracy: 0.6411 - val_loss: 1.4480 - val_accuracy: 0.4214
Epoch 7/14
53/53 - 72s - loss: 0.6008 - accuracy: 0.8208 - val_loss: 1.5242 - val_accuracy: 0.4167
Epoch 8/14
53/53 - 72s - loss: 0.3692 - accuracy: 0.8958 - val_loss: 1.6249 - val_accuracy: 0.4190
Epoch 9/14
53/53 - 72s - loss: 0.2418 - accuracy: 0.9185 - val_loss: 1.6940 - val_accuracy: 0.4310
Epoch 10/14
53/53 - 72s - loss: 0.1869 - accuracy: 0.9476 - val_loss: 1.7167 - val_accuracy: 0.4476
Epoch 11/

In [ ]:
# using lstm unit = 64, bs=32, binary crossentroyp
m1 = model_attention_applied_after_lstm()

m1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

m1.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=32, epochs=10, verbose=2)


Epoch 1/10
53/53 - 74s - loss: 0.4506 - accuracy: 0.1601 - val_loss: 0.4506 - val_accuracy: 0.1643
Epoch 2/10
53/53 - 74s - loss: 0.4506 - accuracy: 0.1494 - val_loss: 0.4506 - val_accuracy: 0.1643
Epoch 3/10
53/53 - 74s - loss: 0.4411 - accuracy: 0.2494 - val_loss: 0.4121 - val_accuracy: 0.3119
Epoch 4/10
53/53 - 73s - loss: 0.3938 - accuracy: 0.3536 - val_loss: 0.3979 - val_accuracy: 0.3524
Epoch 5/10
53/53 - 73s - loss: 0.3437 - accuracy: 0.4512 - val_loss: 0.3985 - val_accuracy: 0.3881
Epoch 6/10
53/53 - 73s - loss: 0.2841 - accuracy: 0.5470 - val_loss: 0.3756 - val_accuracy: 0.4333
Epoch 7/10
53/53 - 73s - loss: 0.2234 - accuracy: 0.6768 - val_loss: 0.3608 - val_accuracy: 0.4619
Epoch 8/10
53/53 - 73s - loss: 0.1684 - accuracy: 0.7762 - val_loss: 0.3702 - val_accuracy: 0.5333
Epoch 9/10
53/53 - 73s - loss: 0.1271 - accuracy: 0.8530 - val_loss: 0.3503 - val_accuracy: 0.5595
Epoch 10/10
53/53 - 73s - loss: 0.0923 - accuracy: 0.9042 - val_loss: 0.3375 - val_accuracy: 0.5929


In [ ]:
# so don't increase epochs (its overfitting)
# m1.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']), batch_size=32, epochs=2, verbose=2)


Epoch 1/2
53/53 - 73s - loss: 0.0654 - accuracy: 0.9440 - val_loss: 0.3807 - val_accuracy: 0.5643
Epoch 2/2
53/53 - 73s - loss: 0.0560 - accuracy: 0.9524 - val_loss: 0.3567 - val_accuracy: 0.5810


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def confussion_mat(model):
    # Predict the values from the validation dataset
    Y_pred = model.predict(data['X_test'])
    # Convert predictions classes to one hot vectors 
    Y_pred_classes = np.argmax(Y_pred,axis = 1) 
    # Convert validation observations to one hot vectors
    Y_true = np.argmax(data['y_test'],axis = 1) 
    # compute the confusion matrix
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    # plot the confusion matrix
    f,ax = plt.subplots(figsize=(15, 15))
    sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
def history2(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
confusion_matrix(m1)

In [ ]:
# when batch size = 124
accr = m.evaluate(data["X_test"],data["y_test"])
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

14/14 [==============================] - 5s 352ms/step - loss: 0.3940 - accuracy: 0.3333
Test set
  Loss: 0.394
  Accuracy: 0.333


In [ ]:
# when batch size = 32
accr = m1.evaluate(data["X_test"],data["y_test"])
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

14/14 [==============================] - 5s 353ms/step - loss: 0.3375 - accuracy: 0.5929
Test set
  Loss: 0.338
  Accuracy: 0.593
